In [97]:
import subprocess
import json
from dotenv import load_dotenv
from constants import *
import os
from web3 import Account, middleware, Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from constants import *
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

In [98]:
# Running the local chain
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [99]:
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --coin={coin} --numderive={numderive} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [100]:
coins = {
    ETH: derive_wallets(mnemonic, ETH, 3),
    BTCTEST: derive_wallets(mnemonic, BTCTEST, 3),
    BTC: derive_wallets(mnemonic, BTC, 3)
}
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0xeFb784e5bf469c3758Ef53d4317E6f893831E42A',
   'xprv': 'xprvA3vy8zBCE9ySZaoPyuiwikYXR78XUuexgcd5EituSgoRGQ1LuNUYwBtydKgafZTN3kURMS2s7KJTE4mMFyz64Wr9nMYum69Ftnd4hZGqQjE',
   'xpub': 'xpub6GvKYVi64XXjn4ss5wFx5tVFy8y1tNNp3qYg37JX12LQ9CLVSunoUzDTUcEKYUaY7NPcuZ1NgxhDFQY2p6qJS4VBuRtuEDaFnSWSf9sQ8Z5',
   'privkey': '0x4a6ed997e0629975f11a8440fa17f43e521ceb77c60b3a0b954ad84d34cfb81c',
   'pubkey': '03122fc0e7508a073cb395482099803dbcd2a5ff399870ad2c85823957c9a9c028',
   'pubkeyhash': '7ec5098f86b12161050336b00627b4489d67a243',
   'index': 0},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x1728da2258c0F85b31Ac399c3096D177167a4B50',
   'xprv': 'xprvA3vy8zBCE9ySbirpVZ3ffMYaPcFLTUjuAb7qo8F3CenrGxXWn3j2RYcBFsRQXKcLHkCjvziswoqaZP8ojkXkdCuM7qfKuB4pQxkWrJFu5Zx',
   'xpub': 'xpub6GvKYVi64XXjpCwHbaag2VVJwe5prwTkXp3SbWeekzKq9krfKb3GyLvf77zLVM4LQtPqhznG6Bp72ZtiYE9wvr25ufuTRMLWzgFcG3xcH41',
   'privkey': '0xb856f875d08e2a7406f7176da8e44d51c0f2043db5db75

In [101]:
def priv_key_to_account(coin, priv_key):
    if coin is BTCTEST:
        return PrivateKeyTestnet(priv_key)
    elif coin is ETH:
        return Account.privateKeyToAccount(priv_key)

In [102]:
def create_tx(coin, account, to, amount):
    if coin is BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])
    elif coin is ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, 
             "to": to, 
             "value": amount}
        )
        return {
            'chainId': 888,
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }

In [103]:
def send_tx(coin, account, to, amount):
    tx = create_tx(coin, account, to, amount)
    print(tx)
    signed_tx = account.sign_transaction(tx)
    if coin is BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)
    elif coin is ETH:
        return w3.eth.sendRawTransaction(signed_tx.rawTransaction)

In [106]:
ethtest_account = priv_key_to_account(ETH,coins[ETH][0]['privkey'])

In [107]:
send_tx(ETH,ethtest_account,"0xB6d0101671b9E37aE7702C4B277F9DC2eB668ec9",29000)

{'chainId': 888, 'from': '0xeFb784e5bf469c3758Ef53d4317E6f893831E42A', 'to': '0xB6d0101671b9E37aE7702C4B277F9DC2eB668ec9', 'value': 222, 'gasPrice': 1000000000, 'gas': 21000, 'nonce': 1}


HexBytes('0x38cbbecb54959be58be1f50e2f73e49f04fb74d88a3d2163edcc83190545befa')